In [228]:
#===============================================================================
# class Map
#===============================================================================

class Map:
    def __init__(self,cluster=None):
        self.cluster = cluster

    def permanence(self,p):    # encode permanence
        """
        self.permanence(p): convert permanence to symbolic string
        >>> o = Map()
        >>> o.permanence(0.52)
        'B'
        >>> o.permanence([-1,0,0.01,0.49,0.5,0.99,1,2])
        '<0yaAY1>'
        """
        def upper(x):
            return chr(int(65+(x-0.5)*100//2))
        def lower(x):
            return chr(int(65+32+(0.5-x)*100//2))

        if isinstance(p,list):
            s = ''
            for k in range(len(p)):
                s += self.permanence(p[k])
            return s

        if p < 0:
            return '<'
        elif p == 0:
            return '0'
        elif p == 1:
            return '1'
        elif p > 1:
            return '>'
        elif p < 0.5:
            return lower(p)
        elif p >= 0.5:
            return upper(p)
        else:
            return '?'

    def symbol(self,x):
        """
        self.symbol(x): convert index to symbol or vice versa
        >>> o = Map()
        >>> o.symbol(1)
        'B'
        >>> o.symbol([0,1,25,26,27,51,52])
        'ABZabz52'
        """
        def symb(x):
            if x < 26:
                return chr(65+x)
            elif x < 52:
                return chr(71+x)
            else:
                return str(x)

        if isinstance(x,int):
            return symb(x)
        elif isinstance(x,list):
            s = ''
            for k in range(len(x)):
                s += self.symbol(x[k])
            return s

    def bar(self,n):                   # bar string of length n
            str = ''
            for k in range(n): str += '-'
            return str


In [229]:
o=Map()
print(o.symbol(26))
str(67)
o.symbol([0,1,25,26,27,51,52])
o.bar(5)

a


'-----'

In [233]:
"""
class Cluster(): parameter definition for neurotron clusters
clu = Cluster(m,n,d,s)
"""

from numpy import array

#=========================================================================
# class Cluster
#=========================================================================

class Cluster:
    def __init__(self,m=4,n=10,d=2,s=5,f=10,g=3):
        def _zeros(d,s):
            return [[0 for j in range(s)] for i in range(d)]
        def _tensor(m,n,d,s):
            return [[_zeros(d,s) for j in range(n)] for i in range(m)]
            
        self.shape = (m,n,d,s)
        self.P = _tensor(m,n,d,s)     # prediction permanences
        self.K = _tensor(m,n,d,s)     # prediction indices
        self.F = _tensor(m,n,1,f)     # excitation weights
        self.G = _tensor(m,n,1,g)     # collaboration weights
        
        self.U = array(_zeros(m,n))
        self.Q = array(_zeros(m,n))
        self.D = array(_zeros(m,n))
        self.B = array(_zeros(m,n))
        self.X = array(_zeros(m,n))
        self.Y = array(_zeros(m,n))
        self.S = array(_zeros(m,n))
        self.L = array(_zeros(m,n))

    def kappa(self,i,j=None):
        """
        self.kappa():  convert matrix indices to linear index or vice versa
        >>> o = Cluster(m=4,n=10)
        >>> k = o.kappa(i:=1,j:=3)  # k = i+j*m 
        13
        >>> ij = o.kappa(k:=13)    # i = k%m, j = k//m
        (1,3)
        """

        m,n,d,s = self.shape
        if j is None:
            k = i
            return (k%m,k//m)
        else:
            return i + j*m

    
    def Pmap(self):
            
        def head(n,c):
            str = ''
            for k in range(n): 
                if k == 1:
                    str += '('
                elif k == 2:
                    str += c
                elif k == 3:
                    str += ')'
                else:
                    str += '-'
            return str

        def title(n,x):
            return '-%03g-' % x
            
        def weights(i,j,d):
            m,n,dd,s = self.shape
            map = Map()
            W = self.P[i][j][d]
            str = ''
            for k in range(s):
               str += map.permanence(W[k])
            return str

        def headline(n,s):
            map = Map()
            line = '+'
            for j in range(n):
                #line += title(s,23) + '+'
                line += map.bar(s) + '+'
            return line
                           
        m,n,d,s = self.shape
        head = headline(n,s)
        str = ''
        for i in range(m):
            print(head)
            for mu in range(d):
                line = '|'
                for j in range(n):
                    line += weights(i,j,0) + '|'
                print(line)
        print(head)


In [234]:
clu = Cluster(4,9,2,5)
o=Map(clu)
o.permanence(0.5)
o.permanence([-1,0,0.01,0.49,0.5,0.99,1,2])

'<0yaAY1>'

In [235]:
clu.Pmap()

+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|00000|00000|00000|00000|00000|00000|00000|00000|00000|
|00000|00000|00000|00000|00000|00000|00000|00000|00000|
+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|00000|00000|00000|00000|00000|00000|00000|00000|00000|
|00000|00000|00000|00000|00000|00000|00000|00000|00000|
+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|00000|00000|00000|00000|00000|00000|00000|00000|00000|
|00000|00000|00000|00000|00000|00000|00000|00000|00000|
+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|00000|00000|00000|00000|00000|00000|00000|00000|00000|
|00000|00000|00000|00000|00000|00000|00000|00000|00000|
+-----+-----+-----+-----+-----+-----+-----+-----+-----+
